# <span style="font-size: 28px;"><b>Demo: Extract Hierarchical Data and Prepare It for Integration</b></span>

## **Step 1: Import Necessary Libraries**

In [77]:
from sqlalchemy import create_engine

import pandas as pd

## **Step 2: Connect to SQL Server**

In [78]:
# Connection string components
server = 'DESKTOP'      # Server name
database = 'Products'   # Database name
driver= 'SQL Server'                   

# SQLAlchemy connection string
connection_string = f'mssql+pyodbc://{server}/{database}?driver={driver}&trusted_connection=yes'

# Create the engine
engine = create_engine(connection_string)

## **Step 3:  Loading Data from SQL Server**

In [79]:
# SQL query
query = '''
SELECT
    product_category,
    product_preference,
    COUNT(*) AS preference_count
FROM
    [Products].[dbo].[ProductPreferences]
GROUP BY
    product_category,
    product_preference
ORDER BY
    product_category,
    product_preference;
'''

# Use the engine to connect and execute the query
df = pd.read_sql_query(query, engine)

In [80]:
# Display the DataFrame
df.head(10)

,product_category,product_preference,preference_count
0,Beauty,Haircare,10
1,Beauty,Makeup,10
2,Beauty,Perfume,8
3,Beauty,Skincare,10
4,Clothing,Dresses,10
5,Clothing,Hoodies,8
6,Clothing,Jeans,2
7,Clothing,Shoes,10
8,Clothing,Suits,8
9,Clothing,T-Shirts,2


## **Step 4:** **Handle Missing Values**

In [81]:
# Fill missing numerical values with the median
df['preference_count'] = df['preference_count'].fillna(df['preference_count'].median())

# Fill missing categorical values with the mode (most common category)
df['product_category'] = df['product_category'].fillna(df['product_category'].mode()[0])
df['product_preference'] = df['product_preference'].fillna(df['product_preference'].mode()[0])

## **Step 5:** **Correct Data Types**

In [82]:
# Preference count should be integer
df['preference_count'] = df['preference_count'].astype(int)

# Categorical variables converted to category type to save memory
df['product_category'] = df['product_category'].astype('category')
df['product_preference'] = df['product_preference'].astype('category')

## **Step 6:** **Remove Duplicates**

In [83]:
df = df.drop_duplicates()

## **Step 7:** **Feature Engineering**

In [84]:
# Create a feature indicating high preference (preference_count > 3)
df['high_preference'] = df['preference_count'] > 3

In [85]:
# Display the DataFrame
df.head(10)

,product_category,product_preference,preference_count,high_preference
0,Beauty,Haircare,10,True
1,Beauty,Makeup,10,True
2,Beauty,Perfume,8,True
3,Beauty,Skincare,10,True
4,Clothing,Dresses,10,True
5,Clothing,Hoodies,8,True
6,Clothing,Jeans,2,False
7,Clothing,Shoes,10,True
8,Clothing,Suits,8,True
9,Clothing,T-Shirts,2,False


## **Step 8:** **Export Prepared Data for Power BI**

In [86]:
df.to_csv('preprocessed_for_power_bi.csv', index=False)